# 고수준 API
신경망 모델을 텐서플로를 이용하여 만들었지만, 사실 텐서플로는 더 쉬운 방법을 제공하고 있다.

<br>

* **이전의 코드**

```python
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                   padding='SAME')
```

위의 코드를 **tf.layers** 라는 모듈을 이용하면 조금 더 편하고 쉽게 코드를 작성할 수 있다.

<br>

* **layers를 이용한 코드**

```python
L1 = tf.layers.conv2d(X, 32, [3, 3])
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
```

In [2]:
import tensorflow as tf

# MNIST 데이터셋 사용
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

W0725 00:08:07.964961 4538639808 deprecation.py:323] From <ipython-input-2-0eedb7f90c67>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0725 00:08:07.965787 4538639808 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0725 00:08:07.966721 4538639808 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz


W0725 00:08:08.176733 4538639808 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


---

<br>

* **이전의 코드**

```python
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
```

* **layers를 이용한 코드**

In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

---

<br>

* **이전의 CNN 계층을 만드는 코드**

```python
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                   padding='SAME')
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                   padding='SAME')
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)
```

* **layers를 이용한 코드**

In [6]:
L1 = tf.layers.conv2d(X, 32, [3, 3])
L1 = tf.layers.max_pooling2d(L1, [2, 2], [2, 2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3, 3])
L2 = tf.layers.max_pooling2d(L2, [2, 2], [2, 2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

---

<br>

* **이전의 모델, 손실 함수, 최적화 함수 만드는 코드**

```python
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
```

* **layers를 이용한 코드**

In [8]:
model = tf.layers.dense(L3, 10, activation=None)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
                            logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [9]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                              feed_dict={X: batch_xs,
                                        Y: batch_ys,
                                         # is_training 부분만 바뀜
                                        is_training: True})
        total_cost += cost_val
        
    print('Epoch: ', '%4d' %(epoch + 1),
         'Avg. cost = ', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy,
                       feed_dict={X: mnist.test.images.reshape(
                                                 -1, 28, 28, 1),
                                 Y: mnist.test.labels,
                                  # is_training 부분만 바뀜
                                 is_training: False}))

Epoch:     1 Avg. cost =  0.172
Epoch:     2 Avg. cost =  0.048
Epoch:     3 Avg. cost =  0.031
Epoch:     4 Avg. cost =  0.022
Epoch:     5 Avg. cost =  0.016
Epoch:     6 Avg. cost =  0.015
Epoch:     7 Avg. cost =  0.011
Epoch:     8 Avg. cost =  0.010
Epoch:     9 Avg. cost =  0.008
Epoch:    10 Avg. cost =  0.008
Epoch:    11 Avg. cost =  0.007
Epoch:    12 Avg. cost =  0.008
Epoch:    13 Avg. cost =  0.006
Epoch:    14 Avg. cost =  0.005
Epoch:    15 Avg. cost =  0.005
최적화 완료
정확도:  0.9882
